In [1]:
import yfinance as yf
import pandas as pd
import os
from datetime import datetime
from supabase import create_client, Client
from dotenv import load_dotenv

# 1. 야후 API

In [ ]:
# 추출할 주요 지표들
# values : 티커, en : 티커, ko : 한국어 명칭
tickers_info = {
    '^GSPC': {'en': '^GSPC', 'ko': 'S&P 500 지수'},
    '^NDX': {'en': '^NDX', 'ko': '나스닥 100 지수'},
    '^DJI': {'en': '^DJI', 'ko': '다우존스 산업평균지수'},
    '^KS11': {'en': '^KS11', 'ko': '코스피 지수'},
    '^KQ11': {'en': '^KQ11', 'ko': '코스닥 지수'},
    'LIT': {'en': 'LIT', 'ko': '리튬 ETF'},
    '^TNX': {'en': 'TNX', 'ko': '미국 10년물 국채 수익률'},
    'NBI': {'en': 'NBI', 'ko': '나스닥 바이오테크놀로지 지수'},
    '^VIX': {'en': '^VIX', 'ko': 'CBOE 변동성 지수'},
    'CL=F': {'en': 'CL=F', 'ko': 'WTI 원유 선물'},
    'FDN': {'en': 'FDN', 'ko': '다우존스 인터넷 지수'},
    'USDKRW=X': {'en': 'USDKRW=X', 'ko': '달러/원 환율'}
}

start_date = '2023-07-11' #7월 11일부터 
end_date = '2025-07-11' #현재 작업중인 7월 11일까지

# 모든 Ticker 한번에 불러오기
tickers_list = list(tickers_info.keys())
# API 호출
data_wide = yf.download(tickers_list, start=start_date, end=end_date, auto_adjust=True, progress=False)
# 종가(Close) 데이터만 추출
close_prices = data_wide['Close']

# 데이터 전처리
final_df = close_prices.stack().reset_index()
final_df.columns = ['date', 'index_en', 'value']
ko_map = {key: val['ko'] for key, val in tickers_info.items()}
final_df['index_ko'] = final_df['index_en'].map(ko_map)
final_df.dropna(inplace=True)
final_df = final_df[['index_en', 'index_ko', 'date', 'value']]
final_df['date'] = final_df['date'].dt.strftime('%Y-%m-%dT%H:%M:%S')


# 2. 업로드

In [ ]:
# 환경 변수 사용
# 상위 폴더의 .env 파일 경로를 지정하여 로드
# 그러나 보안을 위해 업로드 하지 않음
load_dotenv(dotenv_path='../.env')
supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)

data_to_upload = final_df.to_dict('records') # 업로드 형식 변환
# financial_indices 테이블에 데이터 업로드
data, count = supabase.table("financial_indices").insert(data_to_upload).execute()